In [1]:
import time
import numpy as np 
from zmqRemoteApi import RemoteAPIClient

In [2]:
def degree_To_radian(degree):
    return (degree * np.pi/180)

def FK(theta, alfa, a, d):
    
    Cos_theta = np.cos(theta)
    Sin_theta = np.sin(theta)
    Cos_alfa = np.cos(alfa)
    Sin_alfa = np.sin(alfa)
    Transformation_mat = np.array([
                                    [Cos_theta,-Sin_theta*Cos_alfa,Sin_theta*Sin_alfa,a*Cos_theta],
                                    [Sin_theta,Cos_theta*Cos_alfa,-Cos_theta*Sin_alfa,a*Sin_theta],
                                    [0,Sin_alfa,Cos_alfa,d],
                                    [0,0,0,1]
                                  ])
    return np.round(Transformation_mat.reshape(4,4),2)

In [16]:
def Calculate_Jacopian_P(Trans_mat_arr):
    
    O_0 = np.zeros([3,1])
    Jp_size = len(Trans_mat_arr)
    Z_0 = np.array([[0,0,1]]).reshape(3,1)
    Z_arr = [z[0:3,2] for z in Trans_mat_arr]
    Z_arr.insert(0,Z_0)
    O_arr = [o[0:3,3] for o in Trans_mat_arr]
    O_arr.insert(0,O_0)
    O_last = O_arr[-1].reshape(3,1)
    jp_mat = np.array([
        [np.cross(Z_arr[i].reshape(3,1) , (O_last-O_arr[i].reshape(3,1)),axis=0) for i in range(Jp_size)]
    ])
    return jp_mat.reshape(3,Jp_size).transpose()
    
def Calculate_Jacopian_r(Trans_mat_arr):
    
    Z_0 = np.array([[0,0,1]])
    Jp_size = len(Trans_mat_arr)
    Z_arr = [Trans_mat_arr[z][0:3,2] for z in range(Jp_size-1)]
    Z_arr.insert(0,Z_0)
    Jr_mat = np.array([[z.reshape(3,1) for z in Z_arr]])
    return Jr_mat.reshape(3,Jp_size).transpose()

def Calculate_J(Trans_mat_arr):
    
    J_size = len(Trans_mat_arr)
    J = np.array([
                    [Calculate_Jacopian_P(Trans_mat_arr),
                     Calculate_Jacopian_r(Trans_mat_arr)]
                ])
    return J.reshape(6,J_size)

In [17]:
def Calculate_0n_Matrix(Trans_mat_arr):
    
    n = len(Trans_mat_arr)
    T_0 = np.identity(4)
    result_mat = []
    for i in range(n):
        T_0 = np.matmul(T_0,Trans_mat_arr[i])
        T_0 = np.round(T_0,2)
        result_mat.append( T_0 )
    return result_mat
    

In [18]:
def Calculate_Delta_X(Td, T0):
    
    Pd = Td[0:3,3]
    P0 = T0[0:3,3]
    Rd = Td[0:3,0:3]
    R0 = T0[0:3,0:3]
    I = np.identity(3)
    Pos_list = np.array([[d-c for d,c in zip(Pd,P0)]])
    Pos_list = Pos_list.reshape(3,1)
    DR = np.matmul(Rd,R0.transpose())
    skew = np.subtract(DR , I)
    skew = np.round(skew,2)
    print("skew = \n",skew)
    vec = np.array([[abs(skew[1,2]),abs(skew[0,2]) ,abs(skew[0,1])]])
    print("vec = \n",vec)
    vec = vec.reshape(3,1)
    Delta_X = np.array([Pos_list,vec])
    return np.round(Delta_X.reshape(6,1),2)  

In [19]:
def Calculate_IK(dX,J,q0):
    # qd = q0 + J+ * d_X
    j_pinv = np.linalg.pinv(J)
    q0_arr = np.array([q0])
    q0_arr = q0_arr.reshape(6,1)
    qd = np.add(q0_arr,np.matmul(np.round(j_pinv,2),dX)) # 6*1
    return np.round(qd.reshape(1,6),2)

In [20]:
client = RemoteAPIClient()
sim = client.getObject('sim')
# When simulation is not running, ZMQ message handling could be a bit
# slow, since the idle loop runs at 8 Hz by default. So let's make
# sure that the idle loop runs at full speed for this program:
defaultIdleFps = sim.getInt32Param(sim.intparam_idle_fps)
sim.setInt32Param(sim.intparam_idle_fps, 0)

1

In [21]:
# set Object name 
joint_names = [f"joint_{i+1}"  for i in range(6)]

joint_handels = [sim.getObjectHandle(joint) for joint in joint_names]
# set Object handel 
EE_handle = sim.getObjectHandle("EE_h")
EE_h_Pos = sim.getObjectPosition(EE_handle,joint_handels[0])


joint_Position = [sim.getJointPosition(handle) for handle in joint_handels]

joint_6_pos = sim.getObjectPosition(joint_handels[-1],joint_handels[0] )

joint_6_pos = np.round(joint_6_pos,2)

# T0
EE_Trans = np.array([ sim.getObjectMatrix( EE_handle ,joint_handels[0]) ])
EE_Trans = np.round(EE_Trans.reshape(3,4),2)

Target_handle = sim.getObjectHandle("Cuboid")

# print("joint_Position = \n",np.round(joint_Position,2))
# print("EE_Trans = \n",np.round(EE_Trans.reshape(3,4),2))
# print("EE_h_Pos = \n",np.round(EE_h_Pos,2))

for handle in joint_handels:
    sim.setObjectInt32Parameter(handle ,2001,~0) #to enable the position control

In [22]:
# init DH-parameters
q0 = [0,0,0,0,0,0]
alpha = [90,0,0,90,-90,0]
D = [0.1519,0,0,0.11235,0.08535,0.0819]
A = [0,-0.24365,-0.21325,0,0,0]


In [23]:
# calc Transformation matrix using FK
Trans_list = [FK(degree_To_radian(q0[i]),degree_To_radian(alpha[i]),A[i],D[i]) for i in range(6)]

T_0n = Calculate_0n_Matrix(Trans_list)

# T_0n = [ np.array([ sim.getObjectMatrix(joint_handels[i],joint_handels[0] )]).reshape(3,4) for i in range(1,6)]

# T_0n.insert(0,np.array([ sim.getObjectMatrix( joint_handels[0] , -1 ) ]).reshape(3,4) )

# for i in range(6):
#      T_0n[i] = np.round(T_0n[i],2)

# print("qd = \n",qd,"\n\n")
print("T_0n[5] = \n",T_0n[-1],"\n\n")
print("EE_Trans = \n",EE_Trans,"\n\n")
print("EE_h_Pos = \n",np.round(EE_h_Pos,2),"\n\n")
print("joint_6_pos = \n",joint_6_pos)
print("t00 = \n",T_0n[0])
# print("delta_X = \n",delta_X)

# for t in Trans_list:
#     print("t = \n",t,"\n")

# calc T disierd
# T_d = FK(degree_To_radian(30),degree_To_radian(45),0,0)
T_d = np.array([ sim.getObjectMatrix( Target_handle ,-1)])
T_d = T_d.reshape(3,4)
T0_6 = EE_Trans

T_0n[5] = 
 [[ 1.    0.    0.   -0.45]
 [ 0.    0.   -1.   -0.19]
 [ 0.    1.    0.    0.06]
 [ 0.    0.    0.    1.  ]] 


EE_Trans = 
 [[ 0.    1.    0.   -0.  ]
 [ 0.   -0.    1.    0.19]
 [ 1.   -0.    0.    0.55]] 


EE_h_Pos = 
 [-0.    0.19  0.55] 


joint_6_pos = 
 [-0.    0.11  0.55]
t00 = 
 [[ 1.    0.    0.    0.  ]
 [ 0.    0.   -1.    0.  ]
 [ 0.    1.    0.    0.15]
 [ 0.    0.    0.    1.  ]]


In [24]:
# claculate Jacopian matrix
J = Calculate_J(T_0n)

# calculate delta X matrix  
delta_X = Calculate_Delta_X(T_d,EE_Trans)

print("delta_X = \n ",delta_X)

# calculate qd using IK 
qd = Calculate_IK(delta_X,J,q0)

print(qd)

print("np.linalg.norm(delta_X) =",np.round(np.linalg.norm(delta_X),2))

# print("np.linalg.norm(delta_X) = \n",np.round(np.linalg.norm(delta_X),2))

Jp_size =  6
skew = 
 [[-1.  0.  1.]
 [ 1. -1.  0.]
 [ 0.  1. -1.]]
vec = 
 [[0. 1. 0.]]
delta_X = 
  [[ 0.2 ]
 [ 0.66]
 [-0.3 ]
 [ 0.  ]
 [ 1.  ]
 [ 0.  ]]
[[ 1.96 -2.88  0.93  0.17  0.    0.  ]]
np.linalg.norm(delta_X) = 1.25


In [25]:
while ~( np.isclose( np.linalg.norm(delta_X),0.1 ,rtol=0.01)  ):
    
    q0 = qd.copy()
    Trans_list.clear()
    Trans_list = [FK(q0[0][i],degree_To_radian(alpha[i]),A[i],D[i]) for i in range(6)]
    #Trans_list = [FK(theta,degree_To_radian(alp),a,d) for theta,alp,d,a in zip(q0[0],alpha,D,A)]
    T_0n = Calculate_0n_Matrix(Trans_list)

#     T_0n.clear()
#     T_0n = [ np.array([ sim.getObjectMatrix(joint_handels[i],joint_handels[0] )]).reshape(3,4) for i in range(1,6)]
#     T_0n.insert(0,np.array([ sim.getObjectMatrix( joint_handels[0] , -1 ) ]).reshape(3,4) )
    J = Calculate_J(T_0n)
    delta_X = Calculate_Delta_X(T_d,T_0n[-1])
    
    print("delta_X = \n",delta_X)
    print("np.linalg.norm(delta_X) =",np.round(np.linalg.norm(delta_X),2))
    
    qd = Calculate_IK(delta_X,J,q0)
    print("qd = \n",qd,"\n\n")    

Jp_size =  6
skew = 
 [[-0.92 -0.2  -0.98]
 [-0.38 -0.09 -0.21]
 [ 0.93  0.38 -1.  ]]
vec = 
 [[0.21 0.98 0.2 ]]
delta_X = 
 [[ 0.12]
 [ 0.56]
 [-0.18]
 [ 0.21]
 [ 0.98]
 [ 0.2 ]]
np.linalg.norm(delta_X) = 1.19
qd = 
 [[-31.21 -12.57  44.32 -26.16  31.93  21.32]] 


Jp_size =  6
skew = 
 [[-1.2   0.35  0.91]
 [-0.95 -0.89 -0.25]
 [-0.19 -0.93 -0.69]]
vec = 
 [[0.25 0.91 0.35]]
delta_X = 
 [[0.7 ]
 [1.13]
 [0.22]
 [0.25]
 [0.91]
 [0.35]]
np.linalg.norm(delta_X) = 1.68
qd = 
 [[-26.95 -16.46  46.06 -25.97  28.78  28.83]] 


Jp_size =  6
skew = 
 [[-1.17  0.97  0.1 ]
 [ 0.25 -1.06  0.96]
 [ 0.94  0.2  -1.23]]
vec = 
 [[0.96 0.1  0.97]]
delta_X = 
 [[0.27]
 [0.98]
 [0.  ]
 [0.96]
 [0.1 ]
 [0.97]]
np.linalg.norm(delta_X) = 1.7
qd = 
 [[ -4.23 -31.84  19.39 -38.71  21.78   9.08]] 


Jp_size =  6
skew = 
 [[-1.56  0.65 -0.5 ]
 [ 0.18 -1.51 -0.84]
 [-0.81 -0.57 -0.84]]
vec = 
 [[0.84 0.5  0.65]]
delta_X = 
 [[-0.07]
 [ 1.3 ]
 [ 0.08]
 [ 0.84]
 [ 0.5 ]
 [ 0.65]]
np.linalg.norm(delta_X) = 1.75
q

qd = 
 [[  178.62 -1203.62  -571.08   161.94  -472.22  4684.33]] 


Jp_size =  6
skew = 
 [[-1.03  0.92 -0.4 ]
 [-0.82 -0.79  0.54]
 [ 0.58  0.35 -0.25]]
vec = 
 [[0.54 0.4  0.92]]
delta_X = 
 [[0.56]
 [0.49]
 [0.02]
 [0.54]
 [0.4 ]
 [0.92]]
np.linalg.norm(delta_X) = 1.36
qd = 
 [[  177.44 -1201.91  -568.8    161.83  -470.19  4683.86]] 


Jp_size =  6
skew = 
 [[-1.87 -0.31  0.41]
 [-0.25 -1.46 -0.87]
 [ 0.45 -0.85 -0.68]]
vec = 
 [[0.87 0.41 0.31]]
delta_X = 
 [[0.06]
 [0.99]
 [0.1 ]
 [0.87]
 [0.41]
 [0.31]]
np.linalg.norm(delta_X) = 1.42
qd = 
 [[  184.68 -1207.58  -575.06   167.34  -481.5   4668.52]] 


Jp_size =  6
skew = 
 [[-0.14  0.25  0.43]
 [-0.49 -0.72  0.82]
 [ 0.08 -0.92 -0.63]]
vec = 
 [[0.82 0.43 0.25]]
delta_X = 
 [[0.04]
 [0.89]
 [0.13]
 [0.82]
 [0.43]
 [0.25]]
np.linalg.norm(delta_X) = 1.32
qd = 
 [[  162.51 -1197.12  -613.92   174.6   -497.65  4597.24]] 


Jp_size =  6
skew = 
 [[-1.44 -0.16  0.89]
 [ 0.38 -0.14  0.35]
 [-0.82  0.49 -1.32]]
vec = 
 [[0.35 0.89 0.16]]


qd = 
 [[  721.77 -1320.03 -1862.24   620.47  1516.14  4949.46]] 


Jp_size =  6
skew = 
 [[-0.24 -0.61  0.21]
 [-0.54 -1.78 -0.3 ]
 [ 0.35  0.11 -1.94]]
vec = 
 [[0.3  0.21 0.61]]
delta_X = 
 [[0.19]
 [0.98]
 [0.  ]
 [0.3 ]
 [0.21]
 [0.61]]
np.linalg.norm(delta_X) = 1.23
qd = 
 [[  719.87 -1320.83 -1851.04   612.74  1524.89  4952.13]] 


Jp_size =  6
skew = 
 [[-1.54 -0.84 -0.06]
 [-0.74 -0.49 -0.45]
 [ 0.4  -0.19 -1.9 ]]
vec = 
 [[0.45 0.06 0.84]]
delta_X = 
 [[0.01]
 [0.67]
 [0.09]
 [0.45]
 [0.06]
 [0.84]]
np.linalg.norm(delta_X) = 1.17
qd = 
 [[  721.64 -1324.27 -1852.63   612.06  1526.3   4949.85]] 


Jp_size =  6
skew = 
 [[-1.41  0.77 -0.5 ]
 [ 0.67 -1.11 -0.73]
 [-0.62 -0.63 -1.47]]
vec = 
 [[0.73 0.5  0.77]]
delta_X = 
 [[0.3 ]
 [1.01]
 [0.56]
 [0.73]
 [0.5 ]
 [0.77]]
np.linalg.norm(delta_X) = 1.67
qd = 
 [[  721.52 -1323.75 -1853.06   616.2   1522.76  4948.27]] 


Jp_size =  6
skew = 
 [[-0.59  0.84  0.38]
 [ 0.01 -1.41  0.92]
 [ 0.92 -0.37 -1.17]]
vec = 
 [[0.92 0.38 0.84]]


qd = 
 [[  718.99 -1272.45 -1255.53   570.36  1351.64  3204.76]] 


Jp_size =  6
skew = 
 [[-0.04  0.27 -0.04]
 [ 0.04 -1.26 -0.96]
 [-0.27  0.93 -1.27]]
vec = 
 [[0.96 0.04 0.27]]
delta_X = 
 [[ 0.53]
 [ 0.52]
 [-0.11]
 [ 0.96]
 [ 0.04]
 [ 0.27]]
np.linalg.norm(delta_X) = 1.25
qd = 
 [[  721.72 -1271.35 -1266.56   570.57  1352.41  3178.69]] 


Jp_size =  6
skew = 
 [[-1.84 -0.3  -0.46]
 [-0.06 -1.78  0.61]
 [-0.54  0.54 -0.35]]
vec = 
 [[0.61 0.46 0.3 ]]
delta_X = 
 [[0.39]
 [0.8 ]
 [0.03]
 [0.61]
 [0.46]
 [0.3 ]]
np.linalg.norm(delta_X) = 1.21
qd = 
 [[  697.96 -1275.03 -1268.83   572.38  1338.97  3153.17]] 


Jp_size =  6
skew = 
 [[-0.07  0.16  0.32]
 [-0.13 -1.67  0.74]
 [ 0.33 -0.73 -1.6 ]]
vec = 
 [[0.74 0.32 0.16]]
delta_X = 
 [[0.35]
 [1.14]
 [0.43]
 [0.74]
 [0.32]
 [0.16]]
np.linalg.norm(delta_X) = 1.51
qd = 
 [[  762.09 -1293.02 -1195.91   597.    1309.54  2998.03]] 


Jp_size =  6
skew = 
 [[-0.75  0.18  0.96]
 [-0.61 -0.19  0.  ]
 [-0.76 -0.58 -0.69]]
vec = 
 [[0.   0.96 0

skew = 
 [[-0.06  0.34  0.01]
 [ 0.07 -1.17 -0.98]
 [-0.33  0.91 -1.19]]
vec = 
 [[0.98 0.01 0.34]]
delta_X = 
 [[0.1 ]
 [0.78]
 [0.18]
 [0.98]
 [0.01]
 [0.34]]
np.linalg.norm(delta_X) = 1.31
qd = 
 [[  864.38 -1741.75  -468.26  2561.5  -1592.25  1053.07]] 


Jp_size =  6
skew = 
 [[-1.56  0.19 -0.8 ]
 [ 0.81 -0.94 -0.57]
 [-0.06 -0.98 -1.18]]
vec = 
 [[0.57 0.8  0.19]]
delta_X = 
 [[0.19]
 [0.8 ]
 [0.16]
 [0.57]
 [0.8 ]
 [0.19]]
np.linalg.norm(delta_X) = 1.3
qd = 
 [[  844.01 -1720.51  -505.56  2577.64 -1604.89  1259.4 ]] 


Jp_size =  6
skew = 
 [[-1.05  0.98  0.11]
 [ 0.13 -0.88 -0.98]
 [-0.99 -0.04 -1.13]]
vec = 
 [[0.98 0.11 0.98]]
delta_X = 
 [[0.15]
 [0.87]
 [0.21]
 [0.98]
 [0.11]
 [0.98]]
np.linalg.norm(delta_X) = 1.66
qd = 
 [[  840.9  -1716.28  -515.45  2573.27 -1607.99  1259.02]] 


Jp_size =  6
skew = 
 [[-0.77  0.3  -0.92]
 [ 0.76 -1.66 -0.02]
 [-0.61 -0.69 -1.37]]
vec = 
 [[0.02 0.92 0.3 ]]
delta_X = 
 [[ 0.42]
 [ 0.83]
 [-0.32]
 [ 0.02]
 [ 0.92]
 [ 0.3 ]]
np.linalg.norm(

...

qd = 
 [[-13575.32   4841.25 -24394.75 -30691.25  11133.78 -23615.75]] 


Jp_size =  6
skew = 
 [[-0.91 -0.02 -1.  ]
 [-0.87 -0.51 -0.09]
 [ 0.49  0.88 -0.98]]
vec = 
 [[0.09 1.   0.02]]
delta_X = 
 [[0.22]
 [0.62]
 [0.01]
 [0.09]
 [1.  ]
 [0.02]]
np.linalg.norm(delta_X) = 1.2
qd = 
 [[-13571.76   4847.88 -24404.98 -30678.34  11118.75 -23612.41]] 


Jp_size =  6
skew = 
 [[-1.29  0.62  0.72]
 [ 0.94 -0.95  0.34]
 [ 0.18  0.78 -1.6 ]]
vec = 
 [[0.34 0.72 0.62]]
delta_X = 
 [[-0.1 ]
 [ 0.92]
 [ 0.21]
 [ 0.34]
 [ 0.72]
 [ 0.62]]
np.linalg.norm(delta_X) = 1.39
qd = 
 [[-13570.95   4842.84 -24412.9  -30675.48  11126.22 -23616.69]] 


Jp_size =  6
skew = 
 [[-1.5  -0.75  0.39]
 [ 0.68 -1.63 -0.34]
 [ 0.51  0.1  -0.16]]
vec = 
 [[0.34 0.39 0.75]]
delta_X = 
 [[-0.04]
 [ 0.81]
 [ 0.02]
 [ 0.34]
 [ 0.39]
 [ 0.75]]
np.linalg.norm(delta_X) = 1.22
qd = 
 [[-13573.89   4845.86 -24420.76 -30674.24  11123.11 -23614.79]] 


Jp_size =  6
skew = 
 [[-0.05 -0.07  0.28]
 [-0.18 -0.4   0.78]
 [-0.22 -0.79 

skew = 
 [[-1.11 -0.52 -0.84]
 [-0.16 -0.15 -0.5 ]
 [ 0.98  0.08 -1.19]]
vec = 
 [[0.5  0.84 0.52]]
delta_X = 
 [[0.16]
 [0.66]
 [0.44]
 [0.5 ]
 [0.84]
 [0.52]]
np.linalg.norm(delta_X) = 1.37
qd = 
 [[-13645.96   3795.79 -23135.85 -30675.18   9726.21 -24705.09]] 


Jp_size =  6
skew = 
 [[-0.8   0.75 -0.62]
 [ 0.33 -0.45  0.76]
 [ 0.92 -0.37 -1.14]]
vec = 
 [[0.76 0.62 0.75]]
delta_X = 
 [[0.23]
 [1.33]
 [0.23]
 [0.76]
 [0.62]
 [0.75]]
np.linalg.norm(delta_X) = 1.84
qd = 
 [[-13645.06   3794.81 -23127.46 -30681.83   9764.78 -24732.35]] 


Jp_size =  6
skew = 
 [[-0.9  -0.44 -0.9 ]
 [ 0.29 -0.13 -0.39]
 [ 0.95 -0.23 -0.79]]
vec = 
 [[0.39 0.9  0.44]]
delta_X = 
 [[-0.16]
 [ 1.21]
 [ 0.25]
 [ 0.39]
 [ 0.9 ]
 [ 0.44]]
np.linalg.norm(delta_X) = 1.65
qd = 
 [[-13617.61   3800.24 -22952.45 -30677.91   9782.61 -24605.89]] 


Jp_size =  6
skew = 
 [[-1.08 -0.67  0.76]
 [ 0.5  -0.37  0.6 ]
 [-0.86  0.42 -0.71]]
vec = 
 [[0.6  0.76 0.67]]
delta_X = 
 [[ 0.32]
 [ 0.66]
 [-0.34]
 [ 0.6 ]
 [ 0.76]


qd = 
 [[-12732.1    2308.01 -25218.88 -29326.78  10658.47 -29346.34]] 


Jp_size =  6
skew = 
 [[-0.47 -0.35  0.77]
 [-0.18 -1.93 -0.29]
 [ 0.82  0.01 -1.56]]
vec = 
 [[0.29 0.77 0.35]]
delta_X = 
 [[0.45]
 [1.  ]
 [0.27]
 [0.29]
 [0.77]
 [0.35]]
np.linalg.norm(delta_X) = 1.44
qd = 
 [[-12740.03   2298.16 -25224.99 -29331.28  10629.63 -29369.62]] 


Jp_size =  6
skew = 
 [[-0.22 -0.27 -0.58]
 [-0.5  -1.8  -0.32]
 [-0.38  0.54 -1.76]]
vec = 
 [[0.32 0.58 0.27]]
delta_X = 
 [[-0.02]
 [ 0.95]
 [ 0.09]
 [ 0.32]
 [ 0.58]
 [ 0.27]]
np.linalg.norm(delta_X) = 1.19
qd = 
 [[-12741.29   2292.66 -25225.5  -29334.1   10631.23 -29376.03]] 


Jp_size =  6
skew = 
 [[-0.64  0.65  0.67]
 [-0.29 -1.61  0.74]
 [ 0.89 -0.46 -1.02]]
vec = 
 [[0.74 0.67 0.65]]
delta_X = 
 [[0.21]
 [1.22]
 [0.  ]
 [0.74]
 [0.67]
 [0.65]]
np.linalg.norm(delta_X) = 1.72
qd = 
 [[-12739.8    2285.96 -25221.05 -29333.62  10621.53 -29372.99]] 


Jp_size =  6
skew = 
 [[-0.15 -0.49 -0.19]
 [ 0.08 -1.25  0.97]
 [-0.52 -0.84 -1.17

qd = 
 [[-12324.02   1595.78 -25253.12 -29468.23  10578.67 -28633.73]] 


Jp_size =  6
skew = 
 [[-0.03  0.19 -0.14]
 [ 0.23 -1.74  0.64]
 [ 0.03 -0.65 -1.76]]
vec = 
 [[0.64 0.14 0.19]]
delta_X = 
 [[-0.13]
 [ 0.62]
 [-0.05]
 [ 0.64]
 [ 0.14]
 [ 0.19]]
np.linalg.norm(delta_X) = 0.93
qd = 
 [[-12323.99   1590.92 -25259.12 -29468.38  10583.11 -28634.57]] 


Jp_size =  6
skew = 
 [[-1.31 -0.62 -0.73]
 [ 0.   -0.23 -0.65]
 [ 0.95 -0.2  -1.25]]
vec = 
 [[0.65 0.73 0.62]]
delta_X = 
 [[-0.03]
 [ 0.67]
 [ 0.55]
 [ 0.65]
 [ 0.73]
 [ 0.62]]
np.linalg.norm(delta_X) = 1.45
qd = 
 [[-12324.28   1590.78 -25261.41 -29469.97  10582.71 -28636.75]] 


Jp_size =  6
skew = 
 [[-0.36 -0.29 -0.73]
 [ 0.14 -1.87  0.48]
 [-0.77 -0.4  -1.51]]
vec = 
 [[0.48 0.73 0.29]]
delta_X = 
 [[0.29]
 [0.77]
 [0.08]
 [0.48]
 [0.73]
 [0.29]]
np.linalg.norm(delta_X) = 1.24
qd = 
 [[-12316.57   1594.43 -25255.38 -29478.94  10588.78 -28620.24]] 


Jp_size =  6
skew = 
 [[-1.95 -0.03  0.33]
 [-0.31 -1.16 -0.94]
 [ 0.08 -0.99

KeyboardInterrupt: 

In [ ]:
# gripper 

#yum_gripper_handle = sim.getObjectHandle("openCloseJoint")

connector=sim.getObject('./attachPoint')
objectSensor=sim.getObject('./attachProxSensor')

#Target_Object_handle = 


index=0
while True: 
    
    shape=sim.getObjects(index,sim.object_shape_type)
    
    if shape==-1:
        break
        
    cond_1 = sim.getObjectInt32Param(shape,sim.shapeintparam_static)
    cond_2 = sim.getObjectInt32Param(shape,sim.shapeintparam_respondable)
    cond_3 = sim.checkProximitySensor(objectSensor,shape)
    
    if ((cond_1==0) and (cond_2!=0) and (cond_3==1)):
         # Ok, we found a non-static respondable shape that was detected
        attachedShape=shape
         # Do the connection:
        sim.setObjectParent(attachedShape,connector,true)
        break
    index +=1


# c) And just before opening the gripper again, detach the previously attached shape:
#sim.setObjectParent(attachedShape,-1,true)


In [13]:
print(q0)
print(qd)

[[ 48311.15 -12324.62 -58170.31  32275.45 -83880.68  29207.89]]
[[ 48311.15 -12324.62 -58170.31  32275.45 -83880.68  29207.89]]


In [ ]:

client.setStepping(False)
sim.startSimulation()


for (handle,degree) in zip(joint_handels,qd[0]):
    sim.startSimulation()
#     # close
#     sim.setJointTargetVelocity(yum_gripper_handle,-0.02)
    sim.setJointTargetPosition(handle, degree ) # degree by radian
    

In [ ]:
#open
#sim.setJointTargetVelocity(yum_gripper_handle,0.02)

In [ ]:
sim.stopSimulation()